<a href="https://colab.research.google.com/github/augutis44/AI-projektas/blob/main/galutinis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importinu reikalingus paketus

In [ ]:
import gzip
import pickle
import time
import os
import numpy as np
from tensorflow import keras
from datetime import datetime
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

Prisijungiu prie drive, kuriame yra flatland_train.data failas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Pakoreguoju faile esančius duomenis, kad su jais būtų lengviau dirbti. Toliau splitinu duomenis į train ir test 67/33 santykiu

In [ ]:
X,y = pickle.load(gzip.open('/content/drive/MyDrive/Projektas/flatland_train.data','rb'))
y[y != 0] -= 2                                    # Fix labels to run from 0 to 4
X = X.reshape(X.shape[0], 50, 50, 1) / 255.0      # Scale input to range [0, 1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

Pirmiausia mėginu modelį, kuris buvo aprašytas paskaitoje . Modelį treniruojame su train data.

In [ ]:
model = keras.models.Sequential()
# Any pre-processing/re-shaping should be implemented using keras
model.add(keras.layers.Flatten(input_shape=[50, 50, 1]))
model.add(keras.layers.Dense(20, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
210/210 [==============================] - 1s 3ms/step - loss: 1.5284 - accuracy: 0.3387
Epoch 2/20
210/210 [==============================] - 1s 3ms/step - loss: 1.3897 - accuracy: 0.4306
Epoch 3/20
210/210 [==============================] - 1s 3ms/step - loss: 1.3078 - accuracy: 0.4646
Epoch 4/20
210/210 [==============================] - 1s 4ms/step - loss: 1.2493 - accuracy: 0.4907
Epoch 5/20
210/210 [==============================] - 1s 4ms/step - loss: 1.2074 - accuracy: 0.4996
Epoch 6/20
210/210 [==============================] - 1s 3ms/step - loss: 1.1751 - accuracy: 0.5096
Epoch 7/20
210/210 [==============================] - 1s 3ms/step - loss: 1.1507 - accuracy: 0.5272
Epoch 8/20
210/210 [==============================] - 1s 3ms/step - loss: 1.1272 - accuracy: 0.5330
Epoch 9/20
210/210 [==============================] - 1s 3ms/step - loss: 1.1071 - accuracy: 0.5452
Epoch 10/20
210/210 [==============================] - 1s 3ms/step - loss: 1.0899 - accuracy: 0.5597

Gaunu 0.6248 tikslumą. Šio modelio nepakanka, kadangi noriu pasiekti >90 tikslumą.

Patobulinu modelį pridėdama layerių prie modelio. Šį modelį toliau treniruoju su train data.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1))   
model.add(keras.layers.Activation("relu"))            
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.BatchNormalization(axis=-1))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 46, 46, 32)       

In [ ]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
210/210 [==============================] - 10s 13ms/step - loss: 1.2134 - accuracy: 0.5315
Epoch 2/20
210/210 [==============================] - 3s 12ms/step - loss: 0.5356 - accuracy: 0.8028
Epoch 3/20
210/210 [==============================] - 3s 12ms/step - loss: 0.2877 - accuracy: 0.8991
Epoch 4/20
210/210 [==============================] - 3s 12ms/step - loss: 0.1194 - accuracy: 0.9637
Epoch 5/20
210/210 [==============================] - 3s 12ms/step - loss: 0.0509 - accuracy: 0.9852
Epoch 6/20
210/210 [==============================] - 3s 12ms/step - loss: 0.0253 - accuracy: 0.9952
Epoch 7/20
210/210 [==============================] - 3s 12ms/step - loss: 0.0149 - accuracy: 0.9963
Epoch 8/20
210/210 [==============================] - 3s 12ms/step - loss: 0.0173 - accuracy: 0.9954
Epoch 9/20
210/210 [==============================] - 2s 12ms/step - loss: 0.0157 - accuracy: 0.9961
Epoch 10/20
210/210 [==============================] - 3s 12ms/step - loss: 0.0160 - accur

Gaunu jau gerą rezultatą. Tikslumas yra 1.00, o loss gaunasi labia mažas.

Bandau dar kitaip pertvarkytą modelį

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1])) 
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))  
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.BatchNormalization(axis=-1))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_3 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                      

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0233 - accuracy: 0.9913
Epoch 2/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0095 - accuracy: 0.9969
Epoch 3/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0089 - accuracy: 0.9976
Epoch 4/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0069 - accuracy: 0.9976
Epoch 5/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0040 - accuracy: 0.9982
Epoch 6/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0044 - accuracy: 0.9982
Epoch 7/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0055 - accuracy: 0.9978
Epoch 8/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0093 - accuracy: 0.9984
Epoch 9/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0043 - accuracy: 0.9982
Epoch 10/10
210/210 [==============================] - 2s 9ms/step - loss: 0.0030 - accuracy: 0.9985

Modelio tikslumą gaunu 97%

In [ ]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on train set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on train set - 97.21%


In [ ]:
model.save('model.h5')

In [ ]:
def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X_test).numpy().argmax(axis=1) == y_test)
    return acc

model = keras.models.load_model('model.h5')
evaluate('/content/drive/MyDrive/Projektas/flatland_train.data', model)

0.9721212121212122